In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
import json

In [2]:
# Настраиваем трансформатор для изображений

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [14]:
# Указываем директорию хранения файлов, распределенных по папкам(папка = класс)

dataset = datasets.ImageFolder(root='dataset', transform=transform)

In [15]:
# Делим данные на тренировочную и валидационную выборки, вручную определив размерность 

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [16]:
# Загружаем данные

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Фиксируем имена классов
class_names = dataset.classes
print("Классы:", class_names)

Классы: ['0.not_erotic', '1.erotic']


In [18]:
# Загружаем предобученную модель
model = models.resnet18(pretrained=True) 

num_classes = 2                         # Количество классов классификации 
num_features = model.fc.in_features     # Количество входных признаков последнего слоя

# Заменяем последний слой для нашей задачи
model.fc = nn.Linear(num_features, num_classes)  

In [19]:
# Используем GPU, если есть

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [20]:
# Обозначаем функцию потерь и оптимизатор

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [21]:
# Функция обучения для одной эпохи. Фиксируем ошибку и метрику точности

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [22]:
# Честные метрики на валидационной выборке, без обновления весов

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [23]:
num_epochs = 5      # Количество эпох
best_val_acc = 0.0  # Переменная для хранения лучшей метрики

for epoch in range(num_epochs): 
    # Процесс обучения для одной эпохи
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.4f}")

    # Сохраним лучшую модель
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("Модель сохранена!")


Epoch 1/5:
  Train Loss: 0.1370, Train Acc: 0.9411
  Val   Loss: 0.0105, Val   Acc: 0.9935
  Модель сохранена!
Epoch 2/5:
  Train Loss: 0.0179, Train Acc: 0.9935
  Val   Loss: 0.0055, Val   Acc: 1.0000
  Модель сохранена!
Epoch 3/5:
  Train Loss: 0.0085, Train Acc: 0.9984
  Val   Loss: 0.0029, Val   Acc: 1.0000
Epoch 4/5:
  Train Loss: 0.0042, Train Acc: 0.9984
  Val   Loss: 0.0017, Val   Acc: 1.0000
Epoch 5/5:
  Train Loss: 0.0012, Train Acc: 1.0000
  Val   Loss: 0.0043, Val   Acc: 1.0000


In [24]:
dataset.class_to_idx

{'0.not_erotic': 0, '1.erotic': 1}

In [25]:
# Сохраним json-файл с соответсвием имен классов и индекса
with open("class_to_idx.json", "w") as f:
    json.dump(dataset.class_to_idx, f)
